## 1) NLP에서의 pre-training
* 사전 훈련된 단어 이베딩이 모든 nlp 실무자의 도구 상자에서 사전 훈련된 언어 모델로 대체되는 것은 시간 문제이다
* 사전 훈련된 워드 임베딩부터 ELMo, 트랜스포머에 이르기까지 어떻게 발전?

### 1. 사전 훈련된 워드 임베딩
* Word2Vec, FastText, GloVE 같은 워드 임베딩 방법론들을 학습
* 어떤 테스크 수행 시 임베딩 사용 방법 2가지
1. 임베딩 층을 랜덤 초기화하여 처음부터 학습
2. 방대한 데이터로 Word2Vec과 같은 임베딩 알고리즘으로 이미 학습된 임베딩 벡터들을 가져와 사용하는 것
* 문제점 존재
  * 하나의 단어가 하나의 벡터값으로 맵핑되므로 **문맥을 고려하지 못한다**는 단점
    * 다의어나 동음이의어 구분 못함
    * 사전 훈련된 언어 모델을 사용하므로서 극복할 수 있었음

### 2. 사전 훈련된 언어 모델
* 2015년 구글의 'Semi-supervised sequence learning'이라는 논문에서 LSTM 언어 모델을 학습하고 나서 이렇게 학습한 LSTM을 텍스트 분류에 추가 학습하는 방법 제안
  1. 우선 LSTM 언어 모델 학습
    * 언어 모델은 주어진 텍스트로부터 이전 단어들로부터 다음 단어를 예측하도록 학습하므로 기본적으로 별도의 레이블이 부착되지 않은 텍스트 데이터로도 학습 가능
    * 이렇게 레이블이 없는 데이터로 학습된 LSTM과 가중치가 랜덤으로 초기화된 LSTM 두 가지를 두고, IMDB 리뷰 데이터를 학습하여 정확도를 테스트한다면 전자의 LSTM이 좀 더 좋은 성능을 얻음
    * 방대한 텍스트로 LSTM 언어 모델을 학습해두고, 이러한 언어 모델을 다른 테스크에 사용하는 방법으로는 앞서 학습한 ELMo와 같은 아이디어도 존재

* **ELMo**: 순방향 언어 모델과 역방향 언어 모델을 각각 따로 학습시킨 후, 이렇게 사전 학습된 언어 모델로부터 임베딩 값을 얻는다
  * 이 임베딩은 문장에 따라서 의미는 다르지만 소리가 같은 단어들이라고 하더라도 임베딩 벡터값이 달라지므로, Word2Vec이나 GloVe 등과 같은 워드 임베딩 문제 해결 가능

* 트랜스포머 등장 후, 언어 모델 학습 시 LSTM이 아닌 트랜스포머를 사용하는 시도 등장
  * 트랜스포머의 디코더는 LSTM 언어 모델처럼 순차적으로 이전 단어들로부터 다음 단어를 예측
    * Open AI는 트랜스포머 디코더로 총 12개의 층을 쌓은 후, 방대한 텍스트 데이터로부터 언어 모델 GPT-1을 만듦
    * GPT-1에 여러 다양한 테스크를 위해 추가 학습을 시켰을 때, 높은 성능을 얻을 수 있음을 보여줌

* 이제 NLP의 트렌드는 사전 훈련된 언어 모델을 만들어, 이를 다른 태스크에 추가 학습하여 높은 성능을 얻는 것으로 넘어오게 되었음
  * 이를 더 잘 하기 위해 기존의 언어 모델의 구조를 바꿔보고자 하는 시도가 생겨남
    * 언어의 문맥이라는 것은 실제로 양방향이 존재. 하지만 이전 단어들로부터 다음 단어를 예측하는 언어 모델의 특성으로 인해 양방향을 사용할 수 없으므로, 그 대안으로 ELMo에서는 단방향과 역방향으로 언어 모델을 분리해서 학습하는 방법 사용
    * 2018년에는 언어 모델의 양방향 학습을 위해 새로운 구조의 언어 모델 탄생, 이는 마스크드 언어 모델

### 3. 마스크드 언어 모델(Masked Language Model)
* 정의: 입력 텍스트의 단어 집합의 15%의 단어를 랜덤으로 마스킹함. 
  * 마스킹: 원래의 단어가 무엇이었는지 모르게 함
  * 인공 신경망에 이렇게 마스킹된 단어들을 예측하도록 함
    * 문장 중간에 구멍을 뚫어놓고, 구멍에 들어갈 단어들을 예측하는 식으로
    

## 2) 버트(Bidirectional Encoder Representations from Transformers, BERT)
* 2018년 구글이 공개한 사전 훈련된 모델

In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 61.6 MB/s 
     |████████████████████████████████| 596 kB 57.6 MB/s 
     |████████████████████████████████| 895 kB 59.5 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### 1. BERT의 개요
* BERT는 트랜스포머를 이용하여 구현되었으며, 위키피디아(25억 단어)와 BooksCorpus(8억 단어)와 같은 레이블이 없는 텍스트 데이터로 사전 훈련된 언어 모델
* 높은 성능 얻을 수 있었던 이유?
  * 레이블이 없는 방대한 데이터로 사전 훈련된 모델을 가지고, 레이블이 있는 다른 작업(Task)에서 추가 훈련과 함께 하이퍼파라미터를 재조정하여 이 모델을 사용하면 성능이 높게 나오는 기존의 사례들을 참고하였기 때문
    * **파인 튜닝(Fine-tuning)**: 다른 작업에 대해서 파라미터 재조정을 위한 추가 훈련 과정

### 2. BERT의 크기
* BERT의 기본 구조는 트랜스포머의 인코더를 쌓아올린 구조
  * Base 버전에서는 총 12개, Large 버전에서는 총 24개를 쌓음
  * Large 버전은 Base 버전보다 d_model의 크기나 셀프 어텐션 헤드(Self Attention Heads)의 수가 더 큼
    * 트랜스포머 인코더 층의 수를 L, d_model의 크기를 D, 셀프 어텐션 헤드의 수를 A라고 했을 때 각각의 크기는 아래와 같음
      * BERT-Base: L=12, D=768, A=12 : 110M개의 파라미터
      * BERT-Large: L=24, D=1024, A=16 : 340M개의 파라미터
    * BERT-base는 BERT보다 앞서 등장한 Open AI GPT-1과 하이퍼파라미터가 동일한데, 이는 BERT 연구진이 직접적으로 GPT-1과 성능을 비교하기 위해서 동등한 크기로 설계함
    * BERT-Large는 최대 성능을 보여주기 위해 만들어진 모델


### 3. BERT의 문맥을 반영한 임베딩(Contextual Embedding)
* BERT는 ELMo나 GPT-1과 마찬가지로 문맥을 반영한 임베딩 사용
* BERT의 입력: 임베딩층을 지난 임베딩 벡터들
  * d_model: 768, 모든 단어들은 768차원의 임베딩 벡터가 되어 BERT의 입력으로 사용됨
  * BERT는 내부적인 연산을 거친 후, 동일하게 각 단어에 대해서 768차원의 벡터 출력
* BERT의 연산을 거친 후의 출력 임베딩: 문장의 문맥을 모두 참고한 문맥을 반영한 임베딩
  * 하나의 단어가 모든 단어를 참고하는 연산은 사실 BERT의 12개의 층에서 전부 이루어지는 연산
  * 그리고 이를 12개의 층을 지난 후에 최종적으로 출력 임베딩을 얻게 되는 것
  * BERT의 첫번째 층의 출력 임베딩은 BERT의 두번째 층에서는 입력 임베딩이 됨
* 그렇다면 BERT는 어떻게 모든 단어들을 참고하여 문맥을 반영한 출력 임베딩을 얻게 되는 것?
  * 답: **셀프 어텐션**
  * BERT는 기본적으로 트랜스포머 인코더를 12번 쌓은 것이므로 내부적으로 각 층마다 멀티 헤드 셀프 어텐션과 포지션 와이즈 피드 포워드 신경망 수행


### 4. BERT의 서브워드 토크나이저: WordPiece
* BERT는 단어보다 더 작은 단위로 쪼개는 서브워드 토크나이저를 사용
* BERT가 사용한 토크나니저는 WordPiece 토크나이저로 14챕터에서 공부한 바이트 페어 인코딩(Byte Pair Encoding, BPE)의 유사 알고리즘
  * 동작 방식은 BPE와 조금 다르지만, 글자로부터 서브워드들을 병합해가는 방식으로 최종 단어 집합(Vocabulary)를 만드는 것은 BPE와 유사
* 서브워드 토크아니저: 기본적으로 자주 등장하는 단어는 그대로 단어 집합에 추가하지만, 자주 등장하지 않는 단어의 경우에는 더 작은 단위인 서브워드로 분리되어 서브워드들이 단어 집합에 추가된다는 아이디어
  * 이렇게 단어 집합이 만들어지고 나면, 이 단어 집합을 기반으로 토큰화 수행
  * 이는 대표적인 서브워드 토크나이저 패키지인 SentencePiece 실습을 통해 이해한 내용
  * BERT의 서브워드 토크나이저도 이와 마찬가지로 동작
* BERT에서 토큰화를 수행하는 방식
  * 준비물: 이미 훈련 데이터로붙 만들어진 단어 집합
    * 1. 토큰이 단어 집합에 존재한다. => 해당 토큰을 분리하지 않는다.
    * 2. 토큰이 단어 집합에 존재하지 않는다. => 해당 토큰을 서브워드로 분리한다. => 해당 토큰의 첫번째 서브워드를 제외한 나머지 서브워드들은 앞에 "##"를 붙인 것을 토큰으로 한다.

  * embeddings: 서브워드 토크나이저가 아니라면 OOV 문제가 발생. 하지만 서브워드 토크나이저의 경우에는 해당 단어가 단어 집합에 존재하지 않았다고 해서, 서브워드 또한 존재하지 않는다는 의미는 아니므로 해당 단어를 더 쪼개려고 시도
    * 만약, BERT의 단어 집합에 em, ##bed, ##ding, #s라는 서브 워드들이 존재한다면, embeddings는 em, ##bed, ##ding, #s로 분리됨
      * ##는 서브워드들은 단어의 중간부터 등장하는 서브워드라는 것을 알려주기 위해서 단어 집합 생성 시 표시해둔 기호. 이런 표시가 있어야만 쉽게 embeddings로 복원 가능



In [3]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") #Bert-base의 토크나이저
result = tokenizer.tokenize('Here is the sentence I want embeddings for.')
print(result)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

['here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.']


In [4]:
# BERT의 단어 집합에 특정 단어가 있는지 조회
print(tokenizer.vocab['here'])

NameError: ignored

In [6]:
# 단어 임베딩 조회
print(tokenizer.vocab['embeddings'])

# 해당 단어가 존재하지 않는다는 의미에서 KeyError가 발생

KeyError: ignored

In [7]:
print(tokenizer.vocab['em'])

7861


In [8]:
print(tokenizer.vocab['##bed'])

8270


In [9]:
print(tokenizer.vocab['##ding'])

4667


In [10]:
print(tokenizer.vocab['##s'])

2015


In [11]:
# 이제는 BERT의 단어 집합 전체를 불러와서 살펴봄
# BERT의 단어 집합을 vocabulary.txt에 저장

with open('vocabulary.txt', 'w') as f:
  for token in tokenizer.vocab.keys():
    f.write(token + '\n')

In [12]:
df = pd.read_fwf('vocabulary.txt', header=None)
df

,0
0,[PAD]
1,[unused0]
2,[unused1]
3,[unused2]
4,[unused3]
...,...
30517,##．
30518,##／
30519,##：
30520,##？


In [13]:
print('단어 집합의 크기 :',len(df))

단어 집합의 크기 : 30522


In [14]:
# 정수로부터 맵핑된 단어 확인 가능
df.loc[4667].values[0]

'##ding'

In [15]:
df.loc[102].values[0]

'[SEP]'

### 5. 포지션 임베딩(Position Embedding)
* 트랜스포머에서는 포지셔널 인코딩이라는 방법을 통해서 단어의 위치 정보 표현
  * 포지셔널 인코딩: 사인 함수와 코사인 함수를 사용하여 위치에 따라 다른 값을 가지는 행렬을 만들어 이를 단어 벡터들과 더하는 방법
  * BERT에서는 이와 유사하지만, 위치 정보를 사인 함수와 코사인 함수로 만드는 것이 아닌, **학습을 통해서 얻는 포지션 임베딩**이라는 방법 사용

* WordPiece Embedding: 우리가 이미 알고 있는 단어 임베딩. 실직적인 입력.
  * 이 입력에 포지션 임베딩을 통해서 **위치 정보 더해주기**
  * 포지션 임베딩 아이디어 굉장히 간단: **위치 정보를 위한 임베딩 층(Embedding layer)을 하나 더 사용**
  * 문장의 길이가 4라면 4개의 포지션 임베딩 벡터를 학습
  * 그리고 BERT의 입력마다 다음과 같이 포지션 임베딩 벡터를 더해주는 것
    * 첫번째 단어의 임베딩 벡터 + 0번 포지션 임베딩 벡터
    * 두번째 단어의 임베딩 벡터 + 1번 포지션 임베딩 벡터
    * 세번째 단어의 임베딩 벡터 + 2번 포지션 임베딩 벡터
    * 네번째 단어의 임베딩 벡터 + 3번 포지션 임베딩 벡터
  * 실제 BERT에서는 문장의 최대 길이를 512. 총 512개의 포지션 임베딩 벡터가 학습됨
  * 결론적으로 현재 설명한 내용을 기준으로는 BERT에서는 총 두 개의 임베딩 층이 사용됨
    * 단어 집합의 크기가 30,522개인 단어 벡터를 위한 임베딩 층
    * 문장의 최대 길이가 512개의 포지션 벡터를 위한 임베딩 층
* 사실 BERT는 세그먼트 임베딩이라는 1개의 임베딩 층 더 사용
### 6. BERT의 Pre-training
* ELMo: 정방향 LSTM과 역방향 LSTM을 각각 훈련시키는 방식으로 양방향 언어 모델 만듦
* GPT-1: 트랜스포머의 디코더를 이전 단어들로부터 다음 단어를 예측하는 방식으로 단방향 언어 모델 만듦
* BERT: 단방향으로 설계된 GPT-1과는 달리, 화살표가 양방향으로 뻗어나감. 이는 Masked Language Model을 통해 양방향성을 얻었기 때문.
  * BERT의 사전 훈련 방법은 크게 두 가지: 
    1. **마스크드 언어 모델 (Masked Language Model)**
    2. **다음 문장 예측 (Next sentence prediction, NSP)**

## 1) Masked Language Model(MLM)
* BERT는 사전 훈련을 위해서 인공 신경망의 입력으로 들어가는 입력 텍스트의 15%의 단어를 랜덤으로 마스킹
  * 그리고 인공 신경망에게 이 가려진 단어들(Masked words)를 예측하도록 함
  * 중간에 단어들에 구멍을 뚫어놓고, 구멍에 들어갈 단어들을 예측하게 하는 식
* 랜덤으로 선택된 15%의 단어들은 다음과 같은 비율로 규칙이 적용됨
  * 80%의 단어들은 [MASK]로 변경됨
  * 10%의 단어들은 랜덤으로 단어가 변경됨
  * 10%의 단어들은 동일하게 둠
* 이렇게 하는 이유: 
  * [MASK]만 사용할 경우에는 [MASK] 토큰이 파인 튜닝 단계에서는 나타나지 않으므로 사전 학습 단계와 파인 튜닝 단계에서의 불일치가 발생하는 문제
    * 이 문제를 완화하기 위해서 랜덤으로 선택된 15%의 단어들의 모든 토큰을 [MASK]로 사용하지 않음
    * 이를 전체 단어 관점에서 그래프를 통해 정리하면 다음과 같음
      * 그래프로 정리 시, 85% 미사용, 12% [MASK]로 변경 후 예측, 1.5% 랜덤으로 변경 후 예측, 1.5% 미변경 후 예측
  
